In [5]:
import numpy as np
from importlib import reload

from astropy import units as u
from astropy import constants as cnst

from astropy.visualization import hist as ahist

In [6]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams, style
rcParams['axes.prop_cycle'] = style.library['seaborn-deep']['axes.prop_cycle']

In [13]:
def los_estimator(r, vs, alpha=1, beta=0, gamma=3):
    rout = np.max(r)
    C = (alpha + gamma - 2*beta) / (3-2*beta) * rout**(1-alpha)
    return (C/cnst.G * np.mean(vs**2  * r**alpha)).to(u.solMass)

In [14]:
los_estimator([1,2,3]*u.kpc, [10,20,30]*u.km/u.s)

<Quantity 371910226.0608006 solMass>